In [ ]:
import os
import json
from matplotlib import pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
from multiprocessing import Pool
import shutil
import copy

In [ ]:
SPLIT = [80, 20]



with open('/home/ubuntu/data/ceratium-2000-small-labels.json', 'r') as label_file:
    labels = json.load(label_file)
    
img_dir = '/home/ubuntu/data/ceratium-2000-small/'

out_img_dir_train = '/home/ubuntu/data/ceratium-2000-small-train/'
out_img_dir_test = '/home/ubuntu/data/ceratium-2000-small-test/'




In [ ]:
for out_img_dir in [out_img_dir_train, out_img_dir_test]:
    if os.path.exists(out_img_dir):
        shutil.rmtree(out_img_dir)
    os.makedirs(out_img_dir)

In [ ]:
annotations_for_imageid = dict()
annotations_by_id = dict()

for ann_ in labels['annotations']:
    annotations_by_id[ann_['id']] = ann_
    try:
        annotations_for_imageid[ann_['image_id']] += [ann_['id']]
#         print(ann_['image_id'], ': ', annotations_for_imageid[ann_['image_id']])
    except KeyError:
        annotations_for_imageid[ann_['image_id']] = [ann_['id']]

In [ ]:
train_size = len(labels['images']) * SPLIT[0] // 100
test_size = len(labels['images']) * SPLIT[1] // 100
assert(train_size + test_size == len(labels['images']))
print('Splitting to {} train and {} test images.'.format(train_size, test_size))



In [ ]:
a = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [ ]:



train_images = []
train_annotations = []
train_labels = copy.deepcopy(labels)

train_ann_id = 0

for new_index, image_index in enumerate(range(0, train_size)):
    img = labels['images'][image_index]
    ann_ids = annotations_for_imageid[img['id']]
    anns = [annotations_by_id[ann_id] for ann_id in ann_ids]

    img['id'] = new_index
    for ann in anns:
        ann['image_id'] = new_index
        ann['id'] = train_ann_id
        train_ann_id += 1
        
    train_images.append(img)
    train_annotations += anns
    
    src = os.path.join(img_dir, img['file_name'])
    dst = os.path.join(out_img_dir_train, img['file_name'])
    shutil.copyfile(src, dst)
    

train_labels['images'] = train_images
train_labels['annotations'] = train_annotations

train_label_out = '/home/ubuntu/data/ceratium-2000-small-train-labels.json'
with open(train_label_out, 'w') as f:
    json.dump(train_labels, f)     


print('Training set with {} images, {} annotations.'.format(len(train_labels['images']), len(train_labels['annotations'])))
    

In [ ]:
test_images = []
test_annotations = []
test_labels = copy.deepcopy(labels)

test_ann_id = 0

for new_index, image_index in enumerate(range(train_size, train_size+test_size)):
# for new_index, image_index in enumerate(range(334, 336)):
    img = labels['images'][image_index]
    ann_ids = annotations_for_imageid[img['id']]
    anns = [annotations_by_id[ann_id] for ann_id in ann_ids]

    img['id'] = new_index
    for ann in anns:
        ann['image_id'] = new_index
        ann['id'] = test_ann_id
        test_ann_id += 1
    test_images.append(img)
    test_annotations += anns
    
    src = os.path.join(img_dir, img['file_name'])
    dst = os.path.join(out_img_dir_test , img['file_name'])
    shutil.copyfile(src, dst)

test_labels['images'] = test_images
test_labels['annotations'] = test_annotations

test_label_out = '/home/ubuntu/data/ceratium-2000-small-test-labels.json'
with open(test_label_out, 'w') as f:
    json.dump(test_labels, f)   

print('Test set with {} images, {} annotations.'.format(len(test_labels['images']), len(test_labels['annotations'])))
